![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [131]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

First, we're going to import some libraries that we're going to use throughout this notebook.

In [132]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor

Now load the dataset, specifying the columns we want to format as dates.

In [133]:
rental_info = pd.read_csv("rental_info.csv", parse_dates=["rental_date", "return_date"])

Calculate rental length in days (int number).

In [134]:
rental_info["rental_length"] = (rental_info["return_date"] - rental_info["rental_date"]).dt.days
rental_info["rental_length_days"] = rental_info["rental_length"].astype(int)

Create dummy variables for special_features using np.where.

In [135]:
rental_info["deleted_scenes"] = np.where(rental_info["special_features"].str.contains("Deleted Scenes", na=False), 1, 0)
rental_info["behind_the_scenes"] = np.where(rental_info["special_features"].str.contains("Behind the Scenes", na=False), 1, 0)


Select the characteristics and the target.

In [136]:
X = rental_info.drop(columns=["rental_length_days", "rental_date", "return_date", "special_features", "rental_length"])
y = rental_info["rental_length_days"]

Split the data into train and test sets.

In [137]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

Identify the most relevant features using Lasso Regression and filter features with coefficients greater than 0.

In [138]:
lasso = Lasso(alpha=0.01, random_state=9) 
lasso.fit(X_train, y_train)
lasso_coef = lasso.coef_

selected_features = X.columns[lasso_coef > 0]
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

Models to evaluate:

In [139]:
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(max_depth=10, min_samples_split=5, random_state=9),
    "Random Forest": RandomForestRegressor(n_estimators=200, max_depth=20, min_samples_split=5, random_state=9)
}

Train and evaluate models with selected features.

In [140]:
best_mse = float("inf")
best_model = None

for name, model in models.items():
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    mse = mean_squared_error(y_test, y_pred)
    print(f"{name} MSE: {mse:.4f}")

    if mse < best_mse:
        best_mse = mse
        best_model = model

param_dist = {
    "n_estimators": [50, 100, 200, 300],
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

random_search = RandomizedSearchCV(RandomForestRegressor(random_state=9), param_distributions=param_dist, n_iter=10, cv=5, random_state=9, n_jobs=-1)
random_search.fit(X_train_selected, y_train)

Linear Regression MSE: 4.8466
Decision Tree MSE: 3.7227
Random Forest MSE: 2.4470


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=9),
                   n_jobs=-1,
                   param_distributions={'max_depth': [None, 10, 20, 30],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 100, 200, 300]},
                   random_state=9)

Evaluate the optimized model:

In [141]:
best_rf_model = random_search.best_estimator_
y_pred = best_rf_model.predict(X_test_selected)
best_mse = mean_squared_error(y_test, y_pred)
best_model = best_rf_model

print(f"Best Model: {best_model} with MSE: {best_mse:.4f}")

Best Model: RandomForestRegressor(max_depth=30, min_samples_split=5, n_estimators=50,
                      random_state=9) with MSE: 2.4211
